In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
from src.preparation import fetch_dataset
from src.modules.visualizer import do_heatmap



In [3]:

import tensorflow as tf
from tensorflow.compat.v1.math import confusion_matrix
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.models import load_model
from src.processing import make_tf_dataset, split_train_test, dataset_to_array

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt



In [4]:
model_af = load_model('../notebook/model_a1_f1_100.h5')
model_ag = load_model('../notebook/model_a1_g1_100.h5')
model_gf = load_model('../notebook/model_f1_g1_100.h5')
model_all = load_model('../notebook/model_512_100.h5')

W0731 13:00:29.528817 4571239872 deprecation.py:323] From /Users/noresources/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
dataset_a = fetch_dataset('dataset_a1_slim.pkl')
dataset_f = fetch_dataset('dataset_f1_slim.pkl')
dataset_g = fetch_dataset('dataset_g1_slim.pkl')
dataset_all = fetch_dataset('dataset_all_slim.pkl')

In [6]:
# This is the size of the window that is fed into the DNN
window_size = 15
# The number of the features present in the dataset
num_of_features = 42
# Number of distinct labels in the output
label_length = 4
# Hyperparameter that defines the number of samples to work through
# before updating the internal model parameters.
batch_size = 25
# Epochs
ep = 100

In [7]:



def list_to_num(el):
    if el == [1,0,0,0]:
        return 0
    elif el == [0,1,0,0]:
        return 1
    elif el == [0,0,1,0]:
        return 2
    elif el == [0,0,0,1]:
        return 3
    else:
        print('Error',el)
        
def test_model(md, dt):
    print('\n')
    data_test = make_tf_dataset(dt, window_size, num_of_features, label_length ).batch(batch_size)
    results = md.evaluate(data_test)
    
    ph = []
    predictions = md.predict(data_test)
    for pred in predictions:
        i = list(pred).index(max(list(pred)))
        ph.append(i)
    # Ground truth
    fm , labels = dataset_to_array(dt)
    # Padding
    ph = ph + [3 for i in range (0, len(labels) - len(ph))]
    
    # -----------------
    labels = [list_to_num(list(x)) for x in labels]
    # CM
    cm = confusion_matrix(labels, ph)
    print('\n',cm)

In [8]:
test_model(model_af, dataset_g)
test_model(model_ag, dataset_f)
test_model(model_gf, dataset_a)

W0731 13:02:36.552551 4571239872 deprecation.py:323] From /Users/noresources/anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:505: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


    725/Unknown - 35s 48ms/step - loss: 0.1132 - accuracy: 0.9773
 [[ 6666   256    24   122]
 [   40 10410     1   330]
 [   11     7   182    66]
 [    0     0     0     0]]


   2283/Unknown - 94s 41ms/step - loss: 0.0876 - accuracy: 0.9785- 91s 40ms/step -
 [[12828    81   112   340]
 [  158 22035    24   266]
 [   84    22  3347   406]
 [  243   402   344 16387]]


    340/Unknown - 17s 49ms/step - loss: 0.0263 - accuracy: 0.9968
 [[4655   13    0]
 [  42 3775   14]
 [   0    0    0]]


In [9]:
test = dataset_all[-1000:]

In [10]:
test_model(model_all, test)



   2889/Unknown - 316s 109ms/step - loss: 0.0824 - accuracy: 0.9866
 [[20733    82    32   523]
 [   52 31321     0   431]
 [   28    18  2952   408]
 [  148   109   127 15272]]
